In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from data_utils import FaceData
import time
import os

from classifiers.segmentation_bis import SegmentationNN

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

plt.ion()   # interactive mode

In [2]:
train_data = FaceData(image_paths_file='LAG/train/train.txt')
val_data = FaceData(image_paths_file='LAG/val/val.txt')
print(train_data[0][1].size)
print(train_data[0][0].size())

<built-in method size of torch.FloatTensor object at 0x7efc8e79b508>
torch.Size([3, 200, 200])


In [ ]:
print("Train size: %i" % len(train_data))
print("Validation size: %i" % len(val_data))
print("Img size: ", train_data[0][0].size())
print("Segmentation size: ", train_data[0][1].size())

num_example_imgs = 9
plt.figure(figsize=(10, 5 * num_example_imgs))
for i, (img, target) in enumerate(train_data[:num_example_imgs]):
    # img
    plt.subplot(num_example_imgs, 2, i * 2 + 1)
    plt.imshow(img.numpy().reshape(200,200), cmap='gray')
    plt.axis('off')
    if i == 0:
        plt.title("Input image")
    
    # target
    plt.subplot(num_example_imgs, 2, i * 2 + 2)
    plt.imshow(target.numpy().reshape(200,200), cmap='gray')
    plt.axis('off')
    if i == 0:
        plt.title("Target image")
plt.show()

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'LAG'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()


/home/suleman/anaconda3/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [4]:
from classifiers.segmentation_nn import SegmentationNN
from solver import Solver
import torch.nn.functional as F
import torchvision.models as models

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=4)
val_loader = torch.utils.data.DataLoader(val_data,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=4)
model = SegmentationNN()
    

solver = Solver(optim_args={"lr": 1e-4,
                            "eps": 1e-8
                            },
                loss_func = torch.nn.CrossEntropyLoss(ignore_index = -1))

solver.train(model, train_loader, val_loader, log_nth=1, num_epochs=80)

START TRAIN.
[Iteration 1/29840] TRAIN loss: 5.649
[Iteration 2/29840] TRAIN loss: 5.639
[Iteration 3/29840] TRAIN loss: 5.635
[Iteration 4/29840] TRAIN loss: 5.629
[Iteration 5/29840] TRAIN loss: 5.632
[Iteration 6/29840] TRAIN loss: 5.618
[Iteration 7/29840] TRAIN loss: 5.624
[Iteration 8/29840] TRAIN loss: 5.636
[Iteration 9/29840] TRAIN loss: 5.600
[Iteration 10/29840] TRAIN loss: 5.629
[Iteration 11/29840] TRAIN loss: 5.622
[Iteration 12/29840] TRAIN loss: 5.634
[Iteration 13/29840] TRAIN loss: 5.614
[Iteration 14/29840] TRAIN loss: 5.601
[Iteration 15/29840] TRAIN loss: 5.598
[Iteration 16/29840] TRAIN loss: 5.610
[Iteration 17/29840] TRAIN loss: 5.589
[Iteration 18/29840] TRAIN loss: 5.567
[Iteration 19/29840] TRAIN loss: 5.604
[Iteration 20/29840] TRAIN loss: 5.557
[Iteration 21/29840] TRAIN loss: 5.572
[Iteration 22/29840] TRAIN loss: 5.574
[Iteration 23/29840] TRAIN loss: 5.551
[Iteration 24/29840] TRAIN loss: 5.560
[Iteration 25/29840] TRAIN loss: 5.562
[Iteration 26/29840] 

[Iteration 209/29840] TRAIN loss: 5.476
[Iteration 210/29840] TRAIN loss: 5.454
[Iteration 211/29840] TRAIN loss: 5.454
[Iteration 212/29840] TRAIN loss: 5.437
[Iteration 213/29840] TRAIN loss: 5.513
[Iteration 214/29840] TRAIN loss: 5.328
[Iteration 215/29840] TRAIN loss: 5.510
[Iteration 216/29840] TRAIN loss: 5.482
[Iteration 217/29840] TRAIN loss: 5.467
[Iteration 218/29840] TRAIN loss: 5.432
[Iteration 219/29840] TRAIN loss: 5.433
[Iteration 220/29840] TRAIN loss: 5.480
[Iteration 221/29840] TRAIN loss: 5.397
[Iteration 222/29840] TRAIN loss: 5.465
[Iteration 223/29840] TRAIN loss: 5.488
[Iteration 224/29840] TRAIN loss: 5.449
[Iteration 225/29840] TRAIN loss: 5.472
[Iteration 226/29840] TRAIN loss: 5.458
[Iteration 227/29840] TRAIN loss: 5.465
[Iteration 228/29840] TRAIN loss: 5.454
[Iteration 229/29840] TRAIN loss: 5.429
[Iteration 230/29840] TRAIN loss: 5.440
[Iteration 231/29840] TRAIN loss: 5.479
[Iteration 232/29840] TRAIN loss: 5.377
[Iteration 233/29840] TRAIN loss: 5.485


RuntimeError: Expected object of type Variable[torch.cuda.LongTensor] but found type Variable[torch.cuda.FloatTensor] for argument #1 'other'